In [1]:
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 36.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
import re
import copy
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm
import os

In [3]:
# # Load the pre-trained BERT model and tokenizer
# device = "cpu"
# device = "cuda:0"
# model_name = 'bert-base-uncased'  # You can use other pre-trained BERT models
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name).to(device)

'''print(tokens)
    return tokens
    if len(tokens) > 510:
        print("Exceeded token length")
        tokens = tokens[:510]

    # Add the special [CLS] and [SEP] tokens
    tokens = ['[CLS]'] + tokens + ['[SEP]']

    # Convert tokens to token IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)'''

     #print("Shape of embeddings array:", token_embeddings.shape)


    # Extract the embeddings for each token (excluding [CLS] and [SEP])
    #token_embeddings = token_embeddings[0][1:-1]

    #word_embeddings = token_embeddings.cpu().numpy()
    # Average the word embeddings to obtain document embeddings
    #document_embedding_mean = torch.sum(token_embeddings, dim=0).cpu().numpy()

In [4]:
def get_embeddings_for_doc(text):
    # Tokenize the text
    token_ids = tokenizer(text, padding = True)['input_ids']
    
    # Convert token IDs to tensor
    tokens_tensor = torch.tensor(token_ids).to(device)
    if tokens_tensor.shape[1] > 510:
        tokens_tensor = torch.cat((tokens_tensor[..., :510], torch.tensor([[102]], device=device).repeat(tokens_tensor.shape[0], 1)), -1)

    # Get the embeddings
    with torch.no_grad():
        outputs = model(tokens_tensor)
        #token_embeddings = outputs['last_hidden_state']  # Output embeddings from the model
        document_embedding = outputs['pooler_output']

    # Convert tensor to numpy array
    document_embedding = document_embedding.cpu().numpy().reshape(-1,768).sum(0)


    # Print the shape of the word embeddings and document embedding
    #print("Shape of doc mean embeddings:", document_embedding_mean.shape)
    #print("Shape of document embedding:", document_embedding.shape)
    return document_embedding


In [5]:
# a = get_embeddings_for_doc(['I am happy', 'I am very happy a a a a a a'])
# a.shape

In [6]:
# torch.tensor([[102]]).repeat(10, 1).shape

In [7]:
# torch.cuda.is_available()

In [8]:
# a = get_embeddings_for_doc(['I am happy', 'I am very happy'])
# a.shape

In [9]:
# get file path as dict[int, list[str]] where 
# key is the year and value is the list of file paths

# break the text into itemized sections
def get_itemized_10k(fname, sections=['business', 'risk', 'mda', 'qmr']):
    '''Extract ITEM from 10k filing text.

    Args:
        fname: str, the file name (ends with .txt)
        sections: list of sections to extract

    Returns:
        itemized_text: dict[str, str], where key is the section name and value is the text
    '''
    
    with open(fname, encoding='utf-8') as f:
        text = f.read()

    def extract_text(text, item_start, item_end):
        '''
        Args:
            text: 10K filing text
            item_start: compiled regex pattern
            item_end: compiled regex pattern
        '''
        item_start = item_start
        item_end = item_end
        starts = [i.start() for i in item_start.finditer(text)]
        ends = [i.start() for i in item_end.finditer(text)]

        # if no matches, return empty string
        if len(starts) == 0 or len(ends) == 0:
            return None

        # get possible start/end positions
        # we may end up with multiple start/end positions, and we'll choose the longest
        # item text.
        positions = list()
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s,e])

        # get the longest item text
        item_length = 0
        item_position = list()
        for p in positions:
            if (p[1]-p[0]) > item_length:
                item_length = p[1]-p[0]
                item_position = p

        item_text = text[item_position[0]:item_position[1]]

        return item_text

    # extract text for each section
    results = {}

    for section in sections:

        # ITEM 1: Business
        # if there's no ITEM 1A then it ends at ITEM 2
        if section == 'business':
            try:
                item1_start = re.compile("i\s?tem[s]?\s*[1I]\s*[\.\;\:\-\_]*\s*\\b|PART\s*[1I]\s\s*[\.\;\:\-\_]*\s*", re.IGNORECASE)
                item1_end = re.compile("item\s*1a\s*[\.\;\:\-\_]*\s*Risk|item\s*2\s*[\.\,\;\:\-\_]*\s*(Desc|Prop)", re.IGNORECASE)
                business_text = extract_text(text, item1_start, item1_end)
                results['business'] = business_text
            except Exception as e:
                results['business'] = ''
                print(f'Error extracting ITEM 1: Business for {fname}')
            
        # ITEM 1A: Risk Factors
        # it ends at ITEM 2
        if section == 'risk':
            try:
                item1a_start = re.compile("(?<!,\s)item\s*1a[\.\;\:\-\_]*\s*Risk", re.IGNORECASE)
                item1a_end = re.compile("item\s*2\s*[\.\;\:\-\_]*\s*(Desc|Prop)|item\s*[1I]\s*[\.\;\:\-\_]*\s*\\b", re.IGNORECASE)
                risk_text = extract_text(text, item1a_start, item1a_end)
                results['risk'] = risk_text
            except Exception as e:
                results['risk'] = ''
                print(f'Error extracting ITEM 1A: Risk Factors for {fname}')
                
        # ITEM 7: Management's Discussion and Analysis of Financial Condition and Results of Operations
        # it ends at ITEM 7A (if it exists) or ITEM 8
        if section == 'mda':
            try:
                item7_start = re.compile("item\s*7\s*[\.\;\:\-\_]*\s*\\bM", re.IGNORECASE)
                item7_end = re.compile("item\s*7a\s*[\.\;\:\-\_]*[\s\n]*Quanti|item\s*8\s*[\.\,\;\:\-\_]*\s*Finan", re.IGNORECASE)
                item7_text = extract_text(text, item7_start, item7_end)
                results['mda'] = item7_text
            except Exception as e:
                results['mda'] = ''
                print(f'Error extracting ITEM 7: MD&A for {fname}')

        # ITEM 7A: Quantitative and Qualitative Disclosures About Market Risk
        # 
        if section == 'qmr':
            try:
                item7a_start = re.compile("item\s*7a\s*[\.\;\:\-\_]*[\s\n]*Quanti", re.IGNORECASE)
                item7a_end = re.compile("item\s*8\s*[\.\,\;\:\-\_]*\s*Finan", re.IGNORECASE)
                item7a_text = extract_text(text, item7a_start, item7a_end)
                results['qmr'] = item7a_text
            except Exception as e:
                results['qmr'] = ''
                print(f'Error extracting ITEM 7A: for {fname}')
    
    return results


def get_itemized_10q(fname, sections=['mda', 'qmr']):
    '''Extract ITEM from 10q filing text.
    Args:
        fname: str, the file name (ends with .txt)
        sections: list of sections to extract
    Returns:
        itemized_text: dict[str, str], where key is the section name and value is the text
    '''
    
    with open(fname, 'r') as f:
        text = f.read()

    def extract_text(text, item_start, item_end):
        '''
        Args:
            text: 10q filing text
            item_start: compiled regex pattern
            item_end: compiled regex pattern
        '''
        item_start = item_start
        item_end = item_end
        starts = [i.start() for i in item_start.finditer(text)]
        ends = [i.start() for i in item_end.finditer(text)]

        # if no matches, return empty string
        if len(starts) == 0 or len(ends) == 0:
            return None

        # get possible start/end positions
        # we may end up with multiple start/end positions, and we'll choose the longest
        # item text.
        positions = list()
        for s in starts:
            control = 0
            for e in ends:
                if control == 0:
                    if s < e:
                        control = 1
                        positions.append([s,e])

        # get the longest item text
        item_length = 0
        item_position = list()
        for p in positions:
            if (p[1]-p[0]) > item_length:
                item_length = p[1]-p[0]
                item_position = p

        item_text = text[item_position[0]:item_position[1]]

        return item_text

    # extract text for each section
    results = {}

    for section in sections:

        # ITEM 2: Management's Discussion and Analysis of Financial Condition and Results of Operations
        # it ends at ITEM 3
        if section == 'mda':
            try:
                item2_start = re.compile("item\s*2\s*[\.\;\:\-\_]*[\s\n]*Man", re.IGNORECASE)
                item2_end = re.compile("item\s*3\s*[\.\;\:\-\_]*[\s\n]*Quanti|item\s*4\s*", re.IGNORECASE)
                item2_text = extract_text(text, item2_start, item2_end)
                results['mda'] = item2_text
            except Exception as e:
                results['mda'] = ''
                print(f'Error extracting ITEM 2: MD&A for {fname}')
        
        if section == 'qmr':
            try:
                item7a_start = re.compile("item\s*3\s*[\.\;\:\-\_]*[\s\n]*Quanti", re.IGNORECASE)
                item7a_end = re.compile("item\s*4", re.IGNORECASE)
                item7a_text = extract_text(text, item7a_start, item7a_end)
                results['qmr'] = item7a_text
            except Exception as e:
                results['qmr'] = ''
                print(f'Error extracting ITEM 7A: for {fname}')


    return results

In [10]:
def get_info_from_file_name(file_path):
    info = file_path.replace('_edgar', '').replace('_data', '').replace('.txt', '').split('_')
    #['20110328', '10-K', '1403720', '0001469299-11-000174'] 
    date = info[0]
    form_type = info[1]
    cik = info[2]
    acc_num = info[3]

    return date, form_type, cik, acc_num

In [11]:
def preprocess_and_get_BERT(doc_dict,keys):
    section_wise_embeddings = {}
    batch_size = 16
    for key in keys:
        # print(key)
        embeddings = np.zeros(shape = (768,))
        #embeddings['mean_embed'] = np.zeros(shape = (768,))
        sent_count = 0
        text = doc_dict[key]
        if text:
            sentences = text.split('\n')
        else:
            sentences = []
        if len(sentences) == 1:
            sentences = sentences[0].split('.')

        sentences = [sent for sent in sentences if len(sent.split(' ')) > 3]
        for batch_idx in range(0, len(sentences), batch_size):
            batch_sentence = sentences[batch_idx : batch_idx + batch_size]
            doc_emb = get_embeddings_for_doc(batch_sentence)
            # print(doc_emb.shape)
            embeddings += doc_emb
        section_wise_embeddings[key] = embeddings
        if len(sentences) == 0:
            continue
        section_wise_embeddings[key] /= len(sentences)

    return section_wise_embeddings

In [12]:
# sections_10k = ['business', 'risk', 'mda', 'qmr']
# sections_10q = ['mda', 'qmr']
# emb_types = ['max_pool', 'mean_embed']
# parquet_path = '/data/workspace_files/bert_parquet_files/'
# for yr in ['2011', '2012', '2013']:
#     #print(yr)
#     for qtr in ['QTR1', 'QTR2', 'QTR3', 'QTR4']:
#         print(f"Running Year {yr}, QTR : {qtr}")
#         #print(qtr)
#         clean_text_folder = '/data/workspace_files/10X_folders/' + yr + '/' + qtr
#         files = [f for f in os.listdir(clean_text_folder)]
#         for idx in range(0, len(files), 500):
#             save_name = parquet_path + yr + '_' + qtr + '_' + str(idx) + '.parquet'
#             if os.path.exists(save_name):
#                 print(f"Exists : {save_name}")
#                 continue
#             df_list = []
#             pbar = tqdm(files[idx:idx + 500])
#             for file_path in pbar:
#                 # print(file_path)
#                 date, form_type, cik, acc_num = get_info_from_file_name(file_path)
#                 if form_type != '10-K' and form_type != '10-Q':
#                     continue
#                 file_path_req = clean_text_folder + '/' + file_path

#                 if form_type == '10-K':
#                     doc_dict = get_itemized_10k(file_path_req)
#                     keys = copy.deepcopy(sections_10k)
#                 else:
#                     doc_dict = get_itemized_10q(file_path_req)
#                     keys = copy.deepcopy(sections_10q)
#                 section_wise_embeddings = preprocess_and_get_BERT(doc_dict,keys)
#                 row = {'Date': date, "Form_Type": form_type, 'CIK': cik, "Accession_Number": acc_num}
#                 for key in keys:
#                     for emb_type in emb_types:
#                         row[key + '_' + emb_type] = section_wise_embeddings[key]
#                 df_list.append(row)
#             df = pd.DataFrame(df_list)
#             df.to_parquet(save_name, index=False)
#             print("Done for", save_name)
#             #break
#         print("Done for qtr", qtr)
#         #break
#     #break
#     print("Done for year", yr)
#     #break

In [13]:
# parquet_path = '/data/workspace_files/bert_parquet_files'
# file_name = f'{2011}_QTR{1}_{500}.parquet'
# file_path_read_req = parquet_path + file_name
# test = pq.read_table(file_path_read_req).to_pandas()

# parquet_path = '/data/workspace_files/bert_parquet_files/all/'
# file_name = f'{2012}.parquet'
# file_path_read_req = parquet_path + file_name
# test = pq.read_table(file_path_read_req).to_pandas()

In [14]:
def create_section_dict_year(year, req_columns, section = None):
    parquet_path = '/data/workspace_files/bert_parquet_files/'
    if not section is None:
        save_name = f'{parquet_path}{section}/{year}.parquet'
    else:
        save_name = f'{parquet_path}all/{year}.parquet'
    if os.path.exists(save_name):
        print(f"Exists : {save_name}")
        return 
    year_df = []
    if not section is None:
        section_col = f'{section}_max_pool'
    for qtr in range(1, 5):
        print(f'Running for QTR {qtr}')
        for file_idx in range(0, 20000, 500):
            file_name = f'{year}_QTR{qtr}_{file_idx}.parquet'
            file_path_read_req = parquet_path + file_name
            try: 
                if not section is None:
                    df = pq.read_table(file_path_read_req, columns = req_columns + [section_col]).to_pandas()
                else:
                    df = pq.read_table(file_path_read_req).to_pandas()
                    df['all'] = [np.zeros(len(df['mda_max_pool'].iloc[0]))]*len(df)
                    for index, row in df.iterrows():
                        count = 0
                        all_val = 0
                        if row['Form_Type'] == '10-Q':
                            mean_cols = [ele + '_max_pool' for ele in ['mda', 'qmr']]
                        else:
                            mean_cols = [ele + '_max_pool' for ele in ['business', 'risk', 'mda', 'qmr']]
                        for col in mean_cols:
                            if row[col] is None or np.array_equal(row[col], np.zeros(768)):
                                continue
                            count += 1
                            all_val += row[col]
                        if count == 0:
                            continue
                        else:
                            df.at[index,'all'] = all_val/count
                    df = df[req_columns + ['all']]
            except Exception as e:
                print(e)
                break
            year_df.append(df)
    
    year_df = pd.concat(year_df, axis=0)
    year_df.to_parquet(save_name, index=False)
    return 

In [15]:
parquet_path = '/data/workspace_files/bert_parquet_files/'
file_name = '2011_QTR1_0.parquet'
file_path_read_req = parquet_path + file_name
test = pq.read_table(file_path_read_req).to_pandas()

In [16]:
req_columns = ['Date', 'Form_Type', 'CIK', 'Accession_Number']
for year in range(2011, 2014):
    for section in ['business', 'risk', 'mda', 'qmr']:
        create_section_dict_year(str(year), req_columns, section=section)
    create_section_dict_year(str(year), req_columns)

Exists : /data/workspace_files/bert_parquet_files/business/2011.parquet
Exists : /data/workspace_files/bert_parquet_files/risk/2011.parquet
Exists : /data/workspace_files/bert_parquet_files/mda/2011.parquet
Exists : /data/workspace_files/bert_parquet_files/qmr/2011.parquet
Running for QTR 1
/data/workspace_files/bert_parquet_files/2011_QTR1_9500.parquet
Running for QTR 2
/data/workspace_files/bert_parquet_files/2011_QTR2_11000.parquet
Running for QTR 3
/data/workspace_files/bert_parquet_files/2011_QTR3_11000.parquet
Running for QTR 4
/data/workspace_files/bert_parquet_files/2011_QTR4_10000.parquet
Exists : /data/workspace_files/bert_parquet_files/business/2012.parquet
Exists : /data/workspace_files/bert_parquet_files/risk/2012.parquet
Exists : /data/workspace_files/bert_parquet_files/mda/2012.parquet
Exists : /data/workspace_files/bert_parquet_files/qmr/2012.parquet
Running for QTR 1
/data/workspace_files/bert_parquet_files/2012_QTR1_9000.parquet
Running for QTR 2
/data/workspace_files